## Web Scraper

##### 🥣 Selenium & BeautifulSoup: Extract info from web sources
##### 🌩 URL Manipulation: Bypass CloudFlare with string formatters
##### 🔁 For Loops: Iterate through nested elements
##### 🧽 Pandas: Clean and manipulate data
##### 📈 Data Aggregation: Final quality check


In [1]:
# **** Import the required packages **** #
from selenium import webdriver  # <---- library
import requests
from webdriver_manager.chrome import ChromeDriverManager  # <---- library
from selenium.webdriver.common.by import By  # <---- library
import chromedriver_autoinstaller
from selenium.webdriver.common import keys
from selenium.webdriver.support.ui import WebDriverWait


from time import sleep  # <---- time delay for loading as to not break the server
from selenium.webdriver.chrome.service import Service  # <---- library

driver_service = Service(
    executable_path='/Users/calebcougle/PY/chromedriver'
)  # <---- establish service with the path of the driver
from selenium.webdriver.common.keys import Keys  # <---- library
import pandas as pd  # <---- library
from bs4 import BeautifulSoup  #<--- This is beautiufl soup to extract the info we need to look better
import os

import pandas as pd
import numpy as np
from datetime import datetime
import random
from datetime import date, timedelta

##### 👉 URL manipulation using string formatters to avoid CloudFlare


In [2]:
# **** the below cell sets up the functions and variables used for the Webscraping **** # 

In [8]:
# **** Variables that define our 'flight' search **** #
# These variables will be used as parameters to manipulate the webJet URL
urldate = ''  #<--- initialize a variable to use later

destination = 'Sydney'
destination_code = 'SYD'
origin = 'Brisbane'
origin_code = 'BNE'
now = datetime.now()


# **** This function will create the date format required for the url **** #
def url_date_maker(
        from_ymd,
        to_ymd):  #<--- This function will take an INT in YYYYMMMDD format
    global urldate  #<--- Assign global varaible
    from_ymd = pd.to_datetime(
        from_ymd, format="%Y%m%d")  #<--- Take int, convert to Time series
    to_ymd = pd.to_datetime(
        to_ymd, format="%Y%m%d")  #<--- Take int, convert to Time series
    days_between = pd.date_range(
        from_ymd, to_ymd - timedelta(days=1),
        freq='d')  #<--- list all the dates between the two
    urldate = days_between.strftime(
        '%Y%m%d'
    )  #<--- Convert date back to a string and assign to web_date Variable


# Call the function with these dates YYYMMDD
url_date_maker(20221116, 2022016)

In [9]:
# **** initialize variables to store data from a function  **** #
now = datetime.now()
lst_prices = []
lst_carrier = []
flight_depart = []
flight_arrive = []
cabin_class = []
departure_date = []
cabin_class = []
time = random.randint(
    4, 8)  #<--- Using this random INT pass into sleep() function

#####  👉  Selenium Web Driver and BeautifulSoup to find elements and extract info from the source
#####  👉 Using for Loops to iterate through BeautifulSoup objects to find nested elements 

In [10]:
# **** initialize variables to store data from a function  **** #


def web_scraper(web_date, desination, destination_code, origin, origin_code):
    global lst_prices  #<-- Assign global variables to access outside function
    global lst_carrier
    global flight_depart
    global flight_arrive
    global departure_date
    global cabin_class
    i = 0  #<-- Use as counter and to iterate through dates in urldate
    days = len(urldate) - 1  #<-- This is to get index
    while i <= days:
        url2 = 'https://services.webjet.com.au/web/flights/matrix/?Adults=1&Children=0&Infants=0&TravelClass=Economy&GeoCategory=AUDomestic&TripType=Oneway&OneWay={origin_code}-{origin_code}-{destination_code}-{destination_code}-{web_date}&CityFrom={origin}&CityTo={destination}&CityCodeFrom={origin_code}&CityCodeTo={destination_code}'.format(
            destination=destination,
            destination_code=destination_code,
            origin=origin,
            origin_code=origin_code,
            web_date=urldate[i])
        driver = webdriver.Chrome(service=driver_service)
        sleep(
            time
        )  #<-- This stops activity, to prevent me from being 'locked out'
        page = driver.get(url2)
        sleep(time)
        flight_boxes = driver.find_elements(By.ID, "main-container")
        sleep(time)
        for WebElement in flight_boxes:
            elementHTML = WebElement.get_attribute('outerHTML')
            elementSoup = BeautifulSoup(elementHTML,
                                        'html.parser')  #<-- Get the HTML
            for date in elementSoup.find_all(
                    "div", {'class': 'departure-info'}
            ):  #<---- This is used to find the information inside each flight, this could all be done maybe without indents
                for resultrow in elementSoup.find_all(
                        "tr", {'class': 'result-row'
                               }):  #<-- ... contains table row results
                    for time_arrive in resultrow.find(
                            "span",
                        {'class': 'time-arrive'
                         }): #<-- ... contains the arrival time
                        for time_depart in resultrow.find(
                                "span",
                            {'class': 'time-depart'
                             }):  #<-- ... contains the departure time
                            for airline in resultrow.find(
                                    "span", {'class': 'small-carrier-logo'
                                             }):  #<-- ... contains the airline
                                for price in resultrow.find_all(
                                        "span",
                                    {"class": "price-text"
                                     }):  #<-- ... contains the price text
                                    for cabin in price.find_parents(
                                            "td",
                                        {"class": "fare-price"
                                         }):  #<-- ... contains fare-type info
                                        lst_prices.append(
                                            str(price.string[1:])
                                        )  #<-- append the price to lst prices variable as a string object, then convert a python string
                                        lst_carrier.append(
                                            str(airline['alt'])
                                        )  #<-- the alt text to string (which contains the airline call sign), then convert to python string and append
                                        flight_depart.append(
                                            str(time_depart.string)
                                        )  #<-- convert the webelement to string, then convert to python string and append to flight_depart variable
                                        flight_arrive.append(
                                            str(time_arrive.string)
                                        )  #<-- ... to flight_arrive variable
                                        departure_date.append(
                                            str(date.string)
                                        )  #<-- ... to flight_arrive variable
                                        cabin_class.append(
                                            str(cabin.attrs['data-testid'])
                                        )  #<-- ... to cabin_class variable
        i += 1  #<-- ... add to counter
        driver.quit(
        )  #<-- ... I had close and start the session to overcome CloudFlare bot managment lol

In [11]:
web_scraper(urldate,'Sydney','SYD','Brisbane','BNE') #<-- ... Call the function with these arguments. NB// They aren't very user friendly parameters, but I fix this)

#####  👉 Quality checking as the program progresses 

In [ ]:

print(lst_prices)
print(lst_prices)
print(lst_carrier)
print(flight_arrive)
print(flight_depart)
print(departure_date)
print(cabin_class)

In [ ]:

len(departure_date)


In [ ]:
# **** Initilise the DF's to store function data **** #
df_cost = pd.DataFrame()  #<-- ... initilise the DF's
df_carrier = pd.DataFrame()
df_cabin = pd.DataFrame()
df_dep_t = pd.DataFrame()
df_arrv_t = pd.DataFrame()
df_dep_date = pd.DataFrame()

#####  👉 Pandas library to store, clean and manipulate Data Frames

In [ ]:
# **** A dataframe creating function  **** #
# I wanted the ability to manipulate and clean data automatically so the program can be used on other routes/ dates etc


def create_dataframes(
    dep_date, dep_t, arrv_t, carrier, cost, cabin
):  #<-- ... these parameters relate directly to the lists created in the scrape function

    global df_cost, df_carrier, df_cabin, df_dep_t, df_arrv_t, df_dep_date, df_flight_data  #<-- ... make the DataFrames global to acess outside the function
    df_cost = pd.DataFrame(cost)
    df_carrier = pd.DataFrame(carrier)
    df_cabin = pd.DataFrame(cabin)
    df_dep_t = pd.DataFrame(dep_t)
    df_arrv_t = pd.DataFrame(arrv_t)
    df_dep_date = pd.DataFrame(dep_date)

    df_flight_data = pd.concat(  #<-- ... Create a Dataframe with all the merged Data from the scrape
        [df_dep_date, df_dep_t, df_arrv_t, df_carrier, df_cabin, df_cost],
        axis=1,  #<-- ... Merge the DataFrames
        ignore_index=True)  #<-- ... this didn't seem to do anything lol

    column_names = {  #<-- ... A dict of column names to pass
        0: 'departure_date',
        1: 'departure_time',
        2: 'arrival_time',
        3: 'airline',
        4: 'fare_type',
        5: 'price ($)'
    }

    df_flight_data.rename(columns=column_names,
                          inplace=True)  #<-- ... Change our DF into

    dates_col = pd.DatetimeIndex(
        df_flight_data['departure_date'])  #<-- ... Convert departure date using DateTimeIndex and store in dates_col variable
    df_flight_data.insert(
        0, 'year', dates_col.year
    )  #<-- ... From the dates_col, take the year and insert it at index 0 ( to make it the 4th col)
    df_flight_data.insert(
        0, 'month', dates_col.month
    )  #<-- ... Take the month and insert it at index 0 (to make it the 3th col)
    df_flight_data.insert(
        0, 'day', dates_col.day
    )  #<-- ... Take the day and insert it at index 0 (to make it the 2nd col)
    df_flight_data.insert(
        0, 'day_in_week', dates_col.day_name()
    )  #<-- ... Take the day of week and insert it at index 0 (to make it the 1st col)

    df_flight_data['arrival_time'] = pd.to_datetime(
        df_flight_data['arrival_time'], format="%I:%M %p"
    ).dt.time  #<--- Take the string arrival time as "Hours, mins, am/pm" and convert to Date Time ".dt.time" is to get rid of the annoying "1900-01-01"
    df_flight_data['departure_time'] = pd.to_datetime(
        df_flight_data['departure_time'], format="%I:%M %p").dt.time

    # **** Clean the data  **** #
    df_flight_data.drop('departure_date', axis=1,
                        inplace=True)  #<--- Drop this column
    df_flight_data.replace(',', '', regex=True,
                           inplace=True)  #<-- remove commas
    df_flight_data.replace('cell-', '', regex=True,
                           inplace=True)  #<-- remove 'cell'
    df_flight_data.replace(
        'QF', 'Qantas', regex=True,
        inplace=True)  #<-- Change call sign to Airline and regex + inplace 🚩🚩🚩
    df_flight_data.replace('VA', 'Virgin', regex=True, inplace=True)  #<-- ...
    df_flight_data.replace('ZL', 'Rex', regex=True, inplace=True)  #<-- ...
    df_flight_data.replace('JQ', 'JetStar', regex=True, inplace=True)  #<-- ...
    df_flight_data = df_flight_data.astype({
        'fare_type': 'string',
        'airline': 'string',
        'price ($)': 'int'
    })  #<--convert using astype to appropriate data structure


create_dataframes(
    departure_date, flight_depart, flight_arrive, lst_carrier, lst_prices,
    cabin_class
)  #<-- Call the function to create dataframes using the arguments that hold the scraped data
print(df_flight_data)
print(df_cost, df_carrier, df_cabin, df_dep_t, df_arrv_t, df_dep_date)

#####  👉 ✈️Pre-flight check of data for accuracy 

In [ ]:
# **** Pre-flight check for quality  **** #
# another function so I could use this multiple times for desired route
# False = PASS
# True = FAIL
def check_results(data_frame):
    print("**** IMPORTANT INFO ****\nFalse = PASSED\nTrue = FAILED\n********")
    print("****Duplicate Check:****\n", data_frame.duplicated().any())  #
    print("****Null Check:****\n", data_frame.isnull().any())  
    print("****NA Check:****\n", data_frame.isna().any())  
    print("****Data Types Info:****\n", data_frame.info())  

In [ ]:
check_results(
    df_flight_data
)  #<--Call function with merged and cleaned dataframe df_flight_data

In [ ]:
# find the median price of fare-type

#####  👉 Preliminary data aggregation to as a quality check

In [ ]:
# Snap shot of the Data Frame
df_flight_data.head(60)

In [ ]:
# Value Count of Fare Type in  % 
df_flight_data['day'].value_counts(normalize=True)*100

In [ ]:
df_flight_data['fare_type'].value_counts(normalize=True)*100

In [ ]:
# Value Count of Departure time in  % 
df_flight_data['departure_time'].value_counts(normalize=True).head(10)*100

In [ ]:
df_flight_data['arrival_time'].value_counts(normalize=True).head(10)*100

In [ ]:
df_flight_data['airline'].value_counts(normalize=True)*100

### Thank-you 👋🏼✨

In [ ]:
def flightDataScrape(web_date):
    url2 = f'https://services.webjet.com.au/web/flights/matrix/?Adults=1&Children=0&Infants=0&TravelClass=Economy&GeoCategory=AUDomestic&TripType=Oneway&OneWay={origin_code}-{origin_code}-{destination_code}-{destination_code}-{web_date}&CityFrom={origin}&CityTo={destination}&CityCodeFrom={origin_code}&CityCodeTo={destination_code}'.format(
    destination=destination,
    destination_code=destination_code,
    origin=origin,
    origin_code=origin_code,
    web_date=web_date)
    counter = 0 
    lst_prices = []
    lst_carrier = []
    flight_depart = []
    flight_arrive = []
    cabin_class = []
    departure_date = []  #<--- these could be dictionaries
    cabin_class = []

    for WebElement in flight_boxes:
        elementHTML = WebElement.get_attribute('outerHTML')
        elementSoup = BeautifulSoup(elementHTML, 'html.parser')

        for date in elementSoup.find_all("div", {'class': 'departure-info'}):  #<---- This is used to find the information inside each flight, this could all be done maybe without indents
            for resultrow in elementSoup.find_all("tr", {'class': 'result-row'}):
                for time_arrive in resultrow.find("span",{'class': 'time-arrive'}):
                    for time_depart in resultrow.find("span",{'class': 'time-depart'}):
                        for airline in resultrow.find("span", {'class': 'small-carrier-logo'}):
                            for price in resultrow.find_all("span", {"class": "price-text"}):
                                for cabin in price.find_parents("td", {"class": "fare-price"}):
                                    lst_prices.append(price.string[1:])  #<--- this was previous way to make the dollar sign go away lol
                                    lst_carrier.append(airline['alt'])
                                    flight_depart.append(time_depart.string)
                                    flight_arrive.append(time_arrive.string)
                                    departure_date.append(str(date.string))
                                    cabin_class.append(cabin.attrs['data-testid'])
                                    sleep(time)
                                    counter += 1
    print(counter)
print("i'm done")

flightDataScrape(20221113)
    